# config 书写形式
实际上对于实验的参数来说，有非常多的插入形式。较为普遍的就是整合之后在train函数前作为训练输出。
我个人会比较偏爱腾讯挑战赛的编写形式。将代码抽出来作为单独的文件加载。同时，文件中也将参数分为几个小类。

## wechat_challenge config
整个大文件包含以下的类别
 - 首先定义文件名称 parser = argparse.ArgumentParser(description="Baseline for Weixin Challenge 2022")
 - 定义随机数seed\dropout
 - 定义数据类型参数train_annotation \ test_annotation \ train_zip_feats \ test_zip_feats \ test_output_csv \ val_ratio \ batch_size \ val_batch_size \ test_batch_size \ prefetch \ num_workers
 - 定义保存类型savemodel_path \ ckpt_file \ best_score
 - 定义学习参数max_epochs \ max_steps \ print_steps \ warmup_steps \ minimum_lr \ learning_rate \ weight_decay \ adam_epsilon
 - 定义更详细的文件参数，包括Bert \ video \ fusion_layer 等，换了新的项目可以使用其他的板子
 - bert参数包括 bert_dir \ bert_cache \ bert_seq_length \ bert_learning_rate \ bert_warmup_steps \ bert_maxup_steps \ bert_hidden_dropout_prob
 - video参数包括 frame_embedding_size \ max_frames \ vlad_cluster_size \ vlad_groups \ vlad_hidden_size \ se_ratio
 - fusion_layer包括fc_size
 最后return parser.parse_args()

 在训练文件中，需要先在头文件from config import parse_args
 基础的训练语句有
 args = parse_args()
 setup_logging()
 setup_device(args)
 setup_seed(args)

In [1]:
import argparse

def parse_args():
    parser = argparse.ArgumentParser(description="Baseline for Weixin Challenge 2022")

    parser.add_argument("--seed", type=int, default=2022, help="random seed.")
    parser.add_argument('--dropout', type=float, default=0.3, help='dropout ratio')

    # ========================= Data Configs ==========================
    parser.add_argument('--train_annotation', type=str, default='C:/wechat_challenge/annotations/labeled_1.json')
    parser.add_argument('--test_annotation', type=str, default='C:/wechat_challenge/annotations/test_a.json')
    parser.add_argument('--train_zip_feats', type=str, default='C:/wechat_challenge/zip_feats/labeled.zip')
    parser.add_argument('--test_zip_feats', type=str, default='C:/wechat_challenge/zip_feats/test_a.zip')
    parser.add_argument('--test_output_csv', type=str, default='C:/github/challenge-main/result.csv')
    parser.add_argument('--val_ratio', default=0.15, type=float, help='split 10 percentages of training data as validation')
    parser.add_argument('--batch_size', default=64, type=int, help="use for training duration per worker")
    parser.add_argument('--val_batch_size', default=256, type=int, help="use for validation duration per worker")
    parser.add_argument('--test_batch_size', default=256, type=int, help="use for testing duration per worker")
    parser.add_argument('--prefetch', default=8, type=int, help="use for training duration per worker")
    parser.add_argument('--num_workers', default=4, type=int, help="num_workers for dataloaders")

    # ======================== SavedModel Configs =========================
    parser.add_argument('--savedmodel_path', type=str, default='C:/github/challenge-main/save/v3')
    parser.add_argument('--ckpt_file', type=str, default='C:/github/challenge-main/save/v1/model_epoch_4_mean_f1_0.6127.bin')
    parser.add_argument('--best_score', default=0.5, type=float, help='save checkpoint if mean_f1 > best_score')

    # ========================= Learning Configs ==========================
    parser.add_argument('--max_epochs', type=int, default=20, help='How many epochs')
    parser.add_argument('--max_steps', default=50000, type=int, metavar='N', help='number of total epochs to run')
    parser.add_argument('--print_steps', type=int, default=20, help="Number of steps to log training metrics.")
    parser.add_argument('--warmup_steps', default=1000, type=int, help="warm ups for parameters not in bert or vit")
    parser.add_argument('--minimum_lr', default=0., type=float, help='minimum learning rate')
    parser.add_argument('--learning_rate', default=5e-5, type=float, help='initial learning rate')
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-6, type=float, help="Epsilon for Adam optimizer.")

    # ========================== Title BERT =============================
    # parser.add_argument('--bert_dir', type=str, default='hfl/chinese-roberta-wwm-ext-large')
    parser.add_argument('--bert_dir', type=str, default='hfl/chinese-macbert-base')
    parser.add_argument('--bert_cache', type=str, default='data/cache')
    parser.add_argument('--bert_seq_length', type=int, default=80)
    parser.add_argument('--bert_learning_rate', type=float, default=3e-5)
    parser.add_argument('--bert_warmup_steps', type=int, default=5000)
    parser.add_argument('--bert_max_steps', type=int, default=30000)
    parser.add_argument("--bert_hidden_dropout_prob", type=float, default=0.1)

    # ========================== Video =============================
    parser.add_argument('--frame_embedding_size', type=int, default=768)
    parser.add_argument('--max_frames', type=int, default=32)
    parser.add_argument('--vlad_cluster_size', type=int, default=64)
    parser.add_argument('--vlad_groups', type=int, default=8)
    parser.add_argument('--vlad_hidden_size', type=int, default=1024, help='nextvlad output size using dense')
    parser.add_argument('--se_ratio', type=int, default=8, help='reduction factor in se context gating')

    # ========================== Fusion Layer =============================
    parser.add_argument('--fc_size', type=int, default=512, help="linear size before final linear")

    return parser.parse_args()


## unifiedparsing
UperNet采用科研训练代码常用的，将argparse丢进main里面。我认为这个并不是一个很好的书写方式，我个人会坚持将argparse丢进config里面的写法
但是这里有一个很好的习惯，就是在输入了超参数之后会跟一个输出。我尝试利用微信挑战赛的方式给这些超参数分一个类。作为semantic segmentation领域超参数的一个参考和汇总。

In [4]:

parser = argparse.ArgumentParser()
    # Model related arguments
parser.add_argument('--id', default='baseline',
                        help="a name for identifying the model")
parser.add_argument('--arch_encoder', default='resnet50',
                        help="architecture of net_encoder")
parser.add_argument('--arch_decoder', default='upernet',
                        help="architecture of net_decoder")
parser.add_argument('--weights_encoder', default='',
                        help="weights to finetune net_encoder")
parser.add_argument('--weights_decoder', default='',
                        help="weights to finetune net_decoder")
parser.add_argument('--fc_dim', default=2048, type=int,
                        help='number of features between encoder and decoder')

    # optimization related arguments
    parser.add_argument('--num_gpus', default=8, type=int,
                        help='number of gpus to use')
    parser.add_argument('--batch_size_per_gpu', default=2, type=int,
                        help='input batch size')
    parser.add_argument('--num_epoch', default=40, type=int,
                        help='epochs to train for')
    parser.add_argument('--start_epoch', default=1, type=int,
                        help='epoch to start training. useful if continue from a checkpoint')
    parser.add_argument('--epoch_iters', default=5000, type=int,
                        help='iterations of each epoch (irrelevant to batch size)')
    parser.add_argument('--optim', default='SGD', help='optimizer')
    parser.add_argument('--lr_encoder', default=2e-2, type=float, help='LR')
    parser.add_argument('--lr_decoder', default=2e-2, type=float, help='LR')
    parser.add_argument('--lr_pow', default=0.9, type=float,
                        help='power in poly to drop LR')
    parser.add_argument('--beta1', default=0.9, type=float,
                        help='momentum for sgd, beta1 for adam')
    parser.add_argument('--weight_decay', default=1e-4, type=float,
                        help='weights regularizer')
    parser.add_argument('--fix_bn', default=0, type=int,
                        help='fix bn params')

    # Data related arguments
    parser.add_argument('--workers', default=16, type=int,
                        help='number of data loading workers')
    parser.add_argument('--imgSize', default=[300,375,450,525,600], nargs='+', type=int,
                        help='input image size of short edge (int or list)')
    parser.add_argument('--imgMaxSize', default=1000, type=int,
                        help='maximum input image size of long edge')
    parser.add_argument('--padding_constant', default=32, type=int,
                        help='maxmimum downsampling rate of the network')
    parser.add_argument('--segm_downsampling_rate', default=4, type=int,
                        help='downsampling rate of the segmentation label')
    parser.add_argument('--random_flip', default=True, type=bool,
                        help='if horizontally flip images when training')

# saving model configs
parser.add_argument('--seed', default=304, type=int, help='manual seed')
parser.add_argument('--ckpt', default='./ckpt',
                        help='folder to output checkpoints')
parser.add_argument('--disp_iter', type=int, default=20,
                        help='frequency to display')

args = parser.parse_args()
print("Input arguments:")